<a href="https://colab.research.google.com/github/manor-s/DS-course-project-4/blob/main/Classification_Project_Manor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [40]:
import pandas as pd
csv_file_path = '/content/drive/My Drive/e-commerce project/Churn_Modelling.csv'
churn = pd.read_csv(csv_file_path)



# Label Description and EDA summary

## Label Description:

* **RowNumber** - Serial Row Number (from 1 to 10000)

* **CustomerId** - Unique Ids for bank customer identification

* **Surname** - Customer's last name (2932 unique values)

* **CreditScore** - Credit score of the customer

* **Geography** - The country from which the customer belongs

* **Gender** - Male or Female

* **Age** - Age of the customer

* **Tenure** - Number of years for which the customer has been with the bank

* **Balance** - Bank balance of the customer

* **NumOfProducts** - Number of bank products the customer is utilising

* **HasCrCard** - Binary Flag for whether the customer holds a credit card with the bank or not

* **IsActiveMember** - Binary Flag for whether the customer is an active member with the bank or not

* **EstimatedSalary** - Estimated salary of the customer in Dollars

* **Exited** - Binary flag 1 if the customer closed account with bank and 0 if the customer is retained



## Observations related to churn behavior:
* No unexplainable outliers are present.
* No evident interrelationship among the numerical variables.
* **Product Count**: Customers with three or more products exhibit a significantly higher churn rate. Excluding this group could be considered during the modeling process.
* **Geography**: German customers show a distinct churn pattern compared to those from France and Spain.
* **Gender**: Female customers have a higher churn rate than male customers.
* **Active Membership**: Active members tend to have a lower churn rate.
* **Balance**: Accounts with a balance of zero behave differently than those with a positive balance, potentially warranting a separate categorical consideration.
* **Credit Score**: Lower credit scores may correlate with increased churn risk.
* **Age**: Churn rates appear notably higher among customers aged 40 to 60.



# Pre-Processing

In [41]:
import numpy as np

## Making a new categorical variable of Zero Balance or Blance Percentile:

# Define minimum and maximum for positive balances
min_balance = churn.loc[churn['Balance'] > 0, 'Balance'].min()
max_balance = churn['Balance'].max()

# Calculate evenly spaced cutoffs (10 intervals) from min_balance to max_balance
cutoffs = np.linspace(min_balance, max_balance, 10)

# Define labels for each decile (10 labels, one for each decile)
decile_labels = [f'Decile {i+1}' for i in range(10)]

# Assign each balance to the appropriate decile based on these evenly spaced cutoffs
# Add an additional bin edge (-np.inf) for zero balances
churn['Balance_Category'] = pd.cut(
  churn['Balance'],
  bins=[-np.inf, 0] + list(cutoffs),  # Add zero as a separate category
  labels=['Zero Balance'] + decile_labels,     # Add "Zero Balance" label
  include_lowest=True
)


## Making Dummies

churn_from_col_4 = churn.iloc[:, 3:]
churn_dummies = pd.get_dummies(churn_from_col_4, drop_first=False)


In [42]:
churn_dummies.info() # describe(include='all').T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CreditScore                    10000 non-null  int64  
 1   Age                            10000 non-null  int64  
 2   Tenure                         10000 non-null  int64  
 3   Balance                        10000 non-null  float64
 4   NumOfProducts                  10000 non-null  int64  
 5   HasCrCard                      10000 non-null  int64  
 6   IsActiveMember                 10000 non-null  int64  
 7   EstimatedSalary                10000 non-null  float64
 8   Exited                         10000 non-null  int64  
 9   Geography_France               10000 non-null  bool   
 10  Geography_Germany              10000 non-null  bool   
 11  Geography_Spain                10000 non-null  bool   
 12  Gender_Female                  10000 non-null  